# The data used for this project's exploratory data analysis has been sourced from:
1) The Department of Transport dataset (UK data) (https://roadtraffic.dft.gov.uk/custom-downloads). 

2) National Highway Traffic Safety Administration API (US data) (https://crashviewer.nhtsa.dot.gov/CrashAPI). 

## Data cleansing of The Department of Transport dataset (UK data):

#### Two reports have been run from The Department of Transport  data source:
1) The first is 'casualties1' which will contain information about the following characteristics surrounding fatal road incidents in the UK: Sex, Age, Road Type, and Weather Condition. 

2) The second is 'casualties2' which will contain information about the following characteristics surrounding fatal road incidents in the UK: Road User, Speed Limit, Accident Month, and Accident Hour.

In [1]:
# importing dependencies
import pandas as pd
import numpy as np
import scipy as st
import matplotlib.pyplot as plt

In [2]:
# defining csv paths
csv_path1 = "source_data/Road_Casualties_1.csv"
csv_path2 = "source_data/Road_Casualties_2.csv"

### Casualties1 Cleanup

In [3]:
# reading csv files
casualties1 = pd.read_csv(csv_path1)
casualties2 = pd.read_csv(csv_path2)

In [4]:
# creating dataframe 1
casualties1_df = pd.DataFrame(casualties1)
casualties1_df

,Accident year,Casualty severity,Casualty sex,Casualty age,Road type,Weather condition,Casualties,Unnamed: 7
0,2017.0,Killed,Male,1.0,Dual carriageway,Fine no high winds,1.0,NaN
1,2017.0,Killed,Male,2.0,Single carriageway,Fine no high winds,1.0,NaN
2,2017.0,Killed,Male,3.0,Single carriageway,Fine no high winds,1.0,NaN
3,2017.0,Killed,Male,4.0,Single carriageway,Fine no high winds,2.0,NaN
4,2017.0,Killed,Male,5.0,Single carriageway,Fine no high winds,2.0,NaN
...,...,...,...,...,...,...,...,...
2887,2021.0,Killed,Female,93.0,Dual carriageway,Raining + high winds,1.0,NaN
2888,2021.0,Killed,Female,93.0,Single carriageway,Fine no high winds,1.0,NaN
2889,2021.0,Killed,Female,94.0,Single carriageway,Fine no high winds,1.0,NaN
2890,2021.0,Killed,Female,97.0,Dual carriageway,Fine no high winds,1.0,NaN


As can be seen from casualties1_df above, there is a unwanted column named 'Unnamed: 7' and a unwanted last record which only contains NaN values, these both need to be removed.

In [5]:
# removing last 2 columns and last row
casualties1_df = casualties1_df.drop(casualties1_df.columns[-2:], axis=1)
casualties1_df = casualties1_df[:-1]
casualties1_df

,Accident year,Casualty severity,Casualty sex,Casualty age,Road type,Weather condition
0,2017.0,Killed,Male,1.0,Dual carriageway,Fine no high winds
1,2017.0,Killed,Male,2.0,Single carriageway,Fine no high winds
2,2017.0,Killed,Male,3.0,Single carriageway,Fine no high winds
3,2017.0,Killed,Male,4.0,Single carriageway,Fine no high winds
4,2017.0,Killed,Male,5.0,Single carriageway,Fine no high winds
...,...,...,...,...,...,...
2886,2021.0,Killed,Female,93.0,Dual carriageway,Fine no high winds
2887,2021.0,Killed,Female,93.0,Dual carriageway,Raining + high winds
2888,2021.0,Killed,Female,93.0,Single carriageway,Fine no high winds
2889,2021.0,Killed,Female,94.0,Single carriageway,Fine no high winds


Now that the uneccessary columns have been removed, the format of the 'Accident year' column and 'Casualty age' needs to also be converted to a integer (whole number). 

In [6]:
# checking data types
print(casualties1_df.dtypes)

Accident year        float64
Casualty severity     object
Casualty sex          object
Casualty age         float64
Road type             object
Weather condition     object
dtype: object


In [7]:
# converting floats to integers
casualties1_df["Accident year"] = casualties1_df["Accident year"].astype(int)
casualties1_df["Casualty age"] = casualties1_df["Casualty age"].astype(int)
print(casualties1_df.dtypes)

Accident year         int32
Casualty severity    object
Casualty sex         object
Casualty age          int32
Road type            object
Weather condition    object
dtype: object


In [8]:
# Final casualties1 (cleaned)
casualties1_df

,Accident year,Casualty severity,Casualty sex,Casualty age,Road type,Weather condition
0,2017,Killed,Male,1,Dual carriageway,Fine no high winds
1,2017,Killed,Male,2,Single carriageway,Fine no high winds
2,2017,Killed,Male,3,Single carriageway,Fine no high winds
3,2017,Killed,Male,4,Single carriageway,Fine no high winds
4,2017,Killed,Male,5,Single carriageway,Fine no high winds
...,...,...,...,...,...,...
2886,2021,Killed,Female,93,Dual carriageway,Fine no high winds
2887,2021,Killed,Female,93,Dual carriageway,Raining + high winds
2888,2021,Killed,Female,93,Single carriageway,Fine no high winds
2889,2021,Killed,Female,94,Single carriageway,Fine no high winds


# Casualties2

In [9]:
# creating dataframe 2
casualties2_df = pd.DataFrame(casualties2)
casualties2_df

,Accident year,Casualty severity,Road user,Speed limit,Accident month,Accident hour,Casualties,Unnamed: 7
0,2017.0,Killed,Pedestrian,1-20 mph,January,14.0,1.0,NaN
1,2017.0,Killed,Pedestrian,1-20 mph,January,21.0,1.0,NaN
2,2017.0,Killed,Pedestrian,1-20 mph,March,12.0,1.0,NaN
3,2017.0,Killed,Pedestrian,1-20 mph,March,22.0,1.0,NaN
4,2017.0,Killed,Pedestrian,1-20 mph,April,2.0,1.0,NaN
...,...,...,...,...,...,...,...,...
6109,2021.0,Killed,Other vehicle,51-60 mph,May,11.0,1.0,NaN
6110,2021.0,Killed,Other vehicle,51-60 mph,July,10.0,1.0,NaN
6111,2021.0,Killed,Other vehicle,51-60 mph,September,21.0,1.0,NaN
6112,2021.0,Killed,Other vehicle,51-60 mph,November,10.0,1.0,NaN


casualties2_df also has a unwanted column named 'Unnamed: 7' and a unwanted last record which only contains NaN values, these both need to be removed.

In [10]:
# removing last 2 columns and last row
casualties2_df = casualties2_df.drop(casualties2_df.columns[-2:], axis=1)
casualties2_df = casualties2_df[:-1]
casualties2_df

,Accident year,Casualty severity,Road user,Speed limit,Accident month,Accident hour
0,2017.0,Killed,Pedestrian,1-20 mph,January,14.0
1,2017.0,Killed,Pedestrian,1-20 mph,January,21.0
2,2017.0,Killed,Pedestrian,1-20 mph,March,12.0
3,2017.0,Killed,Pedestrian,1-20 mph,March,22.0
4,2017.0,Killed,Pedestrian,1-20 mph,April,2.0
...,...,...,...,...,...,...
6108,2021.0,Killed,Other vehicle,51-60 mph,April,16.0
6109,2021.0,Killed,Other vehicle,51-60 mph,May,11.0
6110,2021.0,Killed,Other vehicle,51-60 mph,July,10.0
6111,2021.0,Killed,Other vehicle,51-60 mph,September,21.0


Now that the uneccessary columns have been removed, the format of the 'Accident year' column and 'Accident hour' needs to also be converted to a integer (whole number). 

In [11]:
# checking data types
print(casualties2_df.dtypes)

Accident year        float64
Casualty severity     object
Road user             object
Speed limit           object
Accident month        object
Accident hour        float64
dtype: object


In [12]:
# converting floats to integers
casualties2_df["Accident year"] = casualties2_df["Accident year"].astype(int)
casualties2_df["Accident hour"] = casualties2_df["Accident hour"].astype(int)
print(casualties2_df.dtypes)

Accident year         int32
Casualty severity    object
Road user            object
Speed limit          object
Accident month       object
Accident hour         int32
dtype: object


In [13]:
# final casualties2 (cleaned)
casualties2_df

,Accident year,Casualty severity,Road user,Speed limit,Accident month,Accident hour
0,2017,Killed,Pedestrian,1-20 mph,January,14
1,2017,Killed,Pedestrian,1-20 mph,January,21
2,2017,Killed,Pedestrian,1-20 mph,March,12
3,2017,Killed,Pedestrian,1-20 mph,March,22
4,2017,Killed,Pedestrian,1-20 mph,April,2
...,...,...,...,...,...,...
6108,2021,Killed,Other vehicle,51-60 mph,April,16
6109,2021,Killed,Other vehicle,51-60 mph,May,11
6110,2021,Killed,Other vehicle,51-60 mph,July,10
6111,2021,Killed,Other vehicle,51-60 mph,September,21


## Data cleansing of the National Highway Traffic Safety Administration API (US data):

In [14]:
# importing dependencies
import requests
import json
import csv 

In [15]:
#url = "https://crashviewer.nhtsa.dot.gov/CrashAPI"

#query = "/FARSData/GetFARSData?dataset=Accident&FromYear=2020&ToYear=2021&format=json"

#query_url = f"{url}{query}"

#crash_viewer = requests.get(query_url)
#crash_viewer = crash_viewer.json()
#crash_viewer

In [16]:
csv_path_3 = "source_data/Accidents_2020_2021.csv"
Accidents_2020_2021 = pd.read_csv(csv_path_3)
Accidents_2020_2021

,caseyear,state,st_case,statename,ve_total,ve_forms,pvh_invl,peds,pernotmvit,permvit,...,hosp_mn,hosp_mnname,cf1,cf1name,cf2,cf2name,cf3,cf3name,fatals,drunk_dr
0,2020,1,10001,Alabama,1,1,0,0,0,4,...,99,Unknown EMS Hospital Arrival Time,NaN,NaN,NaN,NaN,NaN,NaN,3,1.0
1,2020,1,10002,Alabama,4,4,0,0,0,6,...,99,Unknown EMS Hospital Arrival Time,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0
2,2020,1,10003,Alabama,2,2,0,0,0,2,...,99,Unknown EMS Hospital Arrival Time,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0
3,2020,1,10004,Alabama,1,1,0,0,0,5,...,99,Unknown EMS Hospital Arrival Time,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0
4,2020,1,10005,Alabama,1,1,0,0,0,1,...,88,Not Applicable (Not Transported),NaN,NaN,NaN,NaN,NaN,NaN,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75997,2021,56,560100,Wyoming,1,1,0,0,0,2,...,88,Not Applicable (Not Transported),NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
75998,2021,56,560101,Wyoming,2,2,0,0,0,2,...,28,28,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
75999,2021,56,560102,Wyoming,1,1,0,0,0,1,...,88,Not Applicable (Not Transported),NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
76000,2021,56,560103,Wyoming,1,1,0,1,1,1,...,88,Not Applicable (Not Transported),NaN,NaN,NaN,NaN,NaN,NaN,1,NaN


In [17]:
Cleaned_df = Accidents_2020_2021[["caseyear", "st_case","statename","persons", "countyname", "monthname", 
                                  "day_week", "hour", "rur_urbname", "routename", "latitude", "longitud", 
                                  "lgt_condname", "weathername", "fatals", "drunk_dr"]]
Cleaned_df

,caseyear,st_case,statename,persons,countyname,monthname,day_week,hour,rur_urbname,routename,latitude,longitud,lgt_condname,weathername,fatals,drunk_dr
0,2020,10001,Alabama,4,ELMORE (51),January,4,2,Rural,County Road,32.433133,-86.094850,Dark - Not Lighted,Clear,3,1.0
1,2020,10002,Alabama,6,JEFFERSON (73),January,5,17,Urban,Local Street - Municipality,33.484658,-86.839544,Dark - Lighted,Rain,1,0.0
2,2020,10003,Alabama,2,SHELBY (117),January,5,14,Rural,State Highway,33.299942,-86.369642,Daylight,Rain,1,0.0
3,2020,10004,Alabama,5,CALHOUN (15),January,6,15,Rural,County Road,33.795072,-85.883486,Daylight,Cloudy,1,0.0
4,2020,10005,Alabama,1,COOSA (37),January,7,0,Rural,County Road,32.848414,-86.083547,Dark - Not Lighted,Rain,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75997,2021,560100,Wyoming,2,JOHNSON (19),November,7,9,Rural,County Road,44.183594,-106.622031,Daylight,Clear,1,NaN
75998,2021,560101,Wyoming,2,BIG HORN (3),December,3,18,Rural,State Highway,44.257797,-107.856219,Dark - Not Lighted,Clear,1,NaN
75999,2021,560102,Wyoming,1,SWEETWATER (37),December,4,10,Rural,Interstate,41.560719,-109.536278,Daylight,Clear,1,NaN
76000,2021,560103,Wyoming,1,LARAMIE (21),December,1,17,Urban,U.S. Highway,41.138878,-104.766653,Dark - Not Lighted,Clear,1,NaN


In [18]:
Cleaned_df.dtypes

caseyear          int64
st_case           int64
statename        object
persons           int64
countyname       object
monthname        object
day_week          int64
hour              int64
rur_urbname      object
routename        object
latitude        float64
longitud        float64
lgt_condname     object
weathername      object
fatals            int64
drunk_dr        float64
dtype: object

In [19]:
Cleaned_df = Cleaned_df.rename(columns={"caseyear":"Year", "st_case": "Case_ID","statename":"State","persons": "Person_Count", 
                                                 "countyname":"County", "monthname": "Month", "day_week":"Day_Name", 
                                                 "hour":"Hour", "rur_urbname": "Rural_Urban", "routename":"Route_Name", 
                                                 "latitude": "Latitude", "longitud":"Longitude", "lgt_condname":"Visability", 
                                                 "weathername":"Weather_Condition", "fatals":"Fatality_Count", "drunk_dr":"Drunk_Driver_Count"})
Cleaned_df

,Year,Case_ID,State,Person_Count,County,Month,Day_Name,Hour,Rural_Urban,Route_Name,Latitude,Longitude,Visability,Weather_Condition,Fatality_Count,Drunk_Driver_Count
0,2020,10001,Alabama,4,ELMORE (51),January,4,2,Rural,County Road,32.433133,-86.094850,Dark - Not Lighted,Clear,3,1.0
1,2020,10002,Alabama,6,JEFFERSON (73),January,5,17,Urban,Local Street - Municipality,33.484658,-86.839544,Dark - Lighted,Rain,1,0.0
2,2020,10003,Alabama,2,SHELBY (117),January,5,14,Rural,State Highway,33.299942,-86.369642,Daylight,Rain,1,0.0
3,2020,10004,Alabama,5,CALHOUN (15),January,6,15,Rural,County Road,33.795072,-85.883486,Daylight,Cloudy,1,0.0
4,2020,10005,Alabama,1,COOSA (37),January,7,0,Rural,County Road,32.848414,-86.083547,Dark - Not Lighted,Rain,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75997,2021,560100,Wyoming,2,JOHNSON (19),November,7,9,Rural,County Road,44.183594,-106.622031,Daylight,Clear,1,NaN
75998,2021,560101,Wyoming,2,BIG HORN (3),December,3,18,Rural,State Highway,44.257797,-107.856219,Dark - Not Lighted,Clear,1,NaN
75999,2021,560102,Wyoming,1,SWEETWATER (37),December,4,10,Rural,Interstate,41.560719,-109.536278,Daylight,Clear,1,NaN
76000,2021,560103,Wyoming,1,LARAMIE (21),December,1,17,Urban,U.S. Highway,41.138878,-104.766653,Dark - Not Lighted,Clear,1,NaN
